In [1]:
import requests
import pandas as pd
import time

In [2]:
# Make a GET request to the API
url = 'https://mchaexpress.com/sensor_data.php'
max_retries = 3
retry_delay = 1  # in seconds

for _ in range(max_retries):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        break
    else:
        print(f"Failed to get data from API. Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
else:
    print("Max retries reached. Could not get data from API.")
    print("Error: ", response.status_code)

if 'df' in locals():
    print(df.head())

    id  temperature  pressure  humidity    gas  altitude   NC10  PM1_0  PM2_5  \
0  747        40.49   1010.78     58.13   1.00     20.75  20.07   2.50   4.27   
1  748        38.18   1010.67     60.42   1.01     21.92  60.91   7.62   8.90   
2  749        38.85   1010.37     52.06   9.55     24.01  39.99   5.00   6.56   
3  750        38.60   1010.37     52.49  11.62     23.84  39.78   4.97   6.57   
4  751        38.61   1010.39     51.76  13.49     23.84  39.21   4.90   6.46   

   PM4_0  PM10  NC1_0  NC2_5  NC4_0  AQI           created_at  AQI_PM2_5  \
0   5.61  6.28  18.03  19.68  19.99   26  2024-04-06 05:55:34         18   
1   9.59  9.94  59.71  60.69  60.87   41  2024-04-06 06:05:34         37   
2   7.61  8.14  38.31  39.67  39.93   34  2024-04-06 06:23:35         27   
3   7.65  8.19  38.06  39.46  39.72   34  2024-04-06 06:23:46         27   
4   7.51  8.03  37.54  38.90  39.15   33  2024-04-06 06:23:55         27   

   AQI_PM10  
0        26  
1        41  
2        34  


In [3]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Assuming df is your DataFrame
# Convert 'Datetime' column to created_at format and set as index
df['created_at'] = pd.to_datetime(df['created_at'])
pd.date_range(start=df['created_at'].min(), periods=len(df), freq='D')


df.drop(columns = ['id'], inplace=True)
# Sort the DataFrame by the index
df.sort_values(by="created_at")

,temperature,pressure,humidity,gas,altitude,NC10,PM1_0,PM2_5,PM4_0,PM10,NC1_0,NC2_5,NC4_0,AQI,created_at,AQI_PM2_5,AQI_PM10
0,40.49,1010.78,58.13,1.00,20.75,20.07,2.50,4.27,5.61,6.28,18.03,19.68,19.99,26,2024-04-06 05:55:34,18,26
1,38.18,1010.67,60.42,1.01,21.92,60.91,7.62,8.90,9.59,9.94,59.71,60.69,60.87,41,2024-04-06 06:05:34,37,41
2,38.85,1010.37,52.06,9.55,24.01,39.99,5.00,6.56,7.61,8.14,38.31,39.67,39.93,34,2024-04-06 06:23:35,27,34
3,38.60,1010.37,52.49,11.62,23.84,39.78,4.97,6.57,7.65,8.19,38.06,39.46,39.72,34,2024-04-06 06:23:46,27,34
4,38.61,1010.39,51.76,13.49,23.84,39.21,4.90,6.46,7.51,8.03,37.54,38.90,39.15,33,2024-04-06 06:23:55,27,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,31.76,1013.40,69.62,3.16,-1.25,43.05,5.39,6.00,6.24,6.36,42.56,42.96,43.03,27,2024-04-09 01:22:49,25,27
166,31.89,1013.37,69.14,3.17,-0.83,44.48,5.57,6.00,6.09,6.14,44.21,44.43,44.47,26,2024-04-09 01:32:49,25,26
167,32.06,1013.33,67.90,3.14,-0.58,37.96,4.75,5.20,5.34,5.41,37.63,37.90,37.94,23,2024-04-09 01:42:49,22,23
168,32.20,1013.31,68.54,3.09,-0.42,39.89,5.00,5.38,5.47,5.51,39.65,39.85,39.88,23,2024-04-09 01:52:50,22,23


In [4]:
import plotly.express as px
# Convert sensor columns to numeric type
sensor_columns = ['temperature', 'pressure', 'humidity', 'gas', 'altitude', 'PM2_5', 'NC2_5', 'AQI']
df[sensor_columns] = df[sensor_columns].apply(pd.to_numeric)

for sensor in sensor_columns:
    fig = px.line(df, x=df.index, y=sensor, title=f'{sensor.capitalize()} Data')
    fig.update_xaxes(title_text='Datetime')
    fig.update_yaxes(title_text=f'{sensor.capitalize()} Value')
    fig.show()

In [5]:
from sklearn.model_selection import train_test_split
from pycaret.regression import *

# Split the data into train and test sets
train, test = train_test_split(df, test_size=0.1, shuffle=False)

# Check shape
print(train.shape, test.shape)

(153, 17) (17, 17)


In [6]:
df_date = pd.DatetimeIndex(df["created_at"])
temperature_df = df_date, df["temperature"]
humidity_df = df_date, df["humidity"]
pressure_df = df_date, df["pressure"]
altitude_df = df_date, df["altitude"]
gas_df = df_date, df["altitude"]
pm_df = df_date, df["PM2_5"]
nc_df = df_date, df["NC2_5"]
aqi_df = df_date, df["AQI"]

In [7]:
# Function to preprocess the date
def round_off_date(dataframe):
    dataframe = dataframe.round("15T")
    dataframe = dataframe.to_list()
    dataframe = pd.Series(dataframe)
    return dataframe

def concat_date_value(datedf, valuedf):
    concat_df = pd.concat([datedf, valuedf], axis=1)
    concat_df = concat_df.rename(columns={0: "Date"})
    concat_df = concat_df.groupby("Date").sum()
    concat_df = concat_df.asfreq("1H")
    return concat_df

In [8]:
df_date = round_off_date(df_date)

In [9]:
temperature_df = concat_date_value(df_date, df["temperature"])
humidity_df = concat_date_value(df_date, df["humidity"])
pressure_df = concat_date_value(df_date, df["pressure"])
altitude_df = concat_date_value(df_date, df["altitude"])
gas_df = concat_date_value(df_date, df["altitude"])
pm_df = concat_date_value(df_date, df["PM2_5"])
nc_df = concat_date_value(df_date, df["NC2_5"])
aqi_df = concat_date_value(df_date, df["AQI"])

In [10]:
temperature_df["temperature"].fillna(method="ffill", inplace=True)
humidity_df["humidity"].fillna(method="ffill", inplace=True)
pressure_df["pressure"].fillna(method="ffill", inplace=True)
altitude_df["altitude"].fillna(method="ffill", inplace=True)
gas_df["altitude"].fillna(method="ffill", inplace=True)
pm_df["PM2_5"].fillna(method="ffill", inplace=True)
nc_df["NC2_5"].fillna(method="ffill", inplace=True)
aqi_df["AQI"].fillna(method="ffill", inplace=True)

In [11]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
tempExp = TSForecastingExperiment()

tempExp.setup(data=temperature_df, fh = 6, session_id = 123)

# Compare models
best_model = tempExp.compare_models()

print(best_model)
tempExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,temperature
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,0.6204,0.3029,3.5550,3.9472,0.0587,0.0538,-1.5311,5.1400
exp_smooth,Exponential Smoothing,0.8526,0.4105,5.0426,5.4190,0.0787,0.0755,-11.3864,0.1233
theta,Theta Forecaster,0.9487,0.4606,5.6611,6.1093,0.0869,0.0847,-14.8118,0.0467
auto_arima,Auto ARIMA,1.2653,0.6300,7.6258,8.3708,0.1157,0.1161,-63.9659,1.9400
arima,ARIMA,1.2894,0.6247,7.7691,8.2796,0.1179,0.1176,-96.4390,0.0867
stlf,STLF,1.3622,0.6889,8.2277,9.1432,0.1244,0.1304,-134.1022,0.0500
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.5248,0.7601,9.3365,10.1646,0.1374,0.1467,-141.0985,0.2433
snaive,Seasonal Naive Forecaster,1.6479,0.9678,10.0056,12.8801,0.1501,0.1811,-380.0564,0.0700
ets,ETS,2.1087,0.9947,12.9285,13.2667,0.1901,0.2192,-388.4914,0.0900
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,2.5710,1.2736,16.1142,17.1858,0.2280,0.2854,-543.4145,0.1800


NaiveForecaster()


In [12]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
humidExp = TSForecastingExperiment()

humidExp.setup(data=humidity_df, fh = 6, session_id = 123)

# Compare models
best_model = humidExp.compare_models()

print(best_model)
humidExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,humidity
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,0.5282,0.3179,7.9600,8.7414,0.0586,0.0645,-2.5191,0.0400
auto_arima,Auto ARIMA,0.7302,0.4689,10.9923,12.9268,0.0832,0.0915,-6369461047091078869741993984.0000,4.9800
arima,ARIMA,1.1344,0.6646,16.5956,18.2507,0.1363,0.1559,-549321176890824898490925056.0000,0.0900
ets,ETS,1.3258,0.7808,20.1719,21.7330,0.1646,0.1841,-320815809658206122462714789888.0000,0.0767
exp_smooth,Exponential Smoothing,1.3504,0.7915,20.5302,22.0272,0.1678,0.1878,-324553252554351445741485948928.0000,0.1600
snaive,Seasonal Naive Forecaster,1.3506,0.7866,19.6461,21.5967,0.1644,0.1968,-849584494072233499507556352.0000,0.0700
stlf,STLF,1.3555,0.7843,20.4155,21.8203,0.1710,0.1854,-331256090153680319176362688512.0000,0.0567
theta,Theta Forecaster,1.3618,0.7902,20.7495,22.0029,0.1695,0.1894,-347182090858264110191075131392.0000,0.0600
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.3651,0.7876,20.5697,21.8549,0.1688,0.1889,-201964977402383838513232609280.0000,0.2067
grand_means,Grand Means Forecaster,1.7938,0.9980,27.3930,27.8152,0.2255,0.2571,-643448520533229377039781330944.0000,0.0467


NaiveForecaster()


In [13]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
pressExp = TSForecastingExperiment()

pressExp.setup(data=pressure_df, fh = 6, session_id = 123)

# Compare models
best_model = pressExp.compare_models()

print(best_model)
pressExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,pressure
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,0.0132,0.0059,2.0906,2.3568,0.0010,0.0010,-2.2792,0.0533
ets,ETS,0.3129,0.1720,50.1470,69.1275,0.0248,0.0254,-29517786609501098356334657536.0000,0.0567
exp_smooth,Exponential Smoothing,0.4814,0.2208,77.3373,88.7308,0.0383,0.0393,-58906247769355843349157773312.0000,0.0900
theta,Theta Forecaster,0.5332,0.2493,85.9777,100.3612,0.0425,0.0439,-61012270328693793460206960640.0000,0.0533
stlf,STLF,0.7177,0.3335,118.6492,136.0913,0.0587,0.0658,-1316314174063197048685985792.0000,0.0667
arima,ARIMA,0.7764,0.3315,127.8524,134.9954,0.0632,0.0672,-803834383831072019769720832.0000,0.1300
auto_arima,Auto ARIMA,0.9676,0.4175,160.0417,170.4345,0.0792,0.0891,-17456724884087950717564747776.0000,2.4167
snaive,Seasonal Naive Forecaster,1.0272,0.5852,169.8533,238.8769,0.0840,0.1117,-35839.8691,0.0633
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.5681,0.6844,255.4489,277.2885,0.1264,0.1416,-257709411207774193399124459520.0000,0.1900
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,2.8474,1.1616,469.9517,473.6634,0.2325,0.3352,-46370624931851369390666678272.0000,0.2667


NaiveForecaster()


In [14]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
altitExp = TSForecastingExperiment()

altitExp.setup(data=altitude_df, fh = 6, session_id = 123)

# Compare models
best_model = altitExp.compare_models()

print(best_model)
altitExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,altitude
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.5562,0.9775,13.9451,15.6457,0.3449,0.3379,-85003055041179295941413306368.0000,0.4433
grand_means,Grand Means Forecaster,1.6265,1.0137,15.1986,16.5317,0.4461,0.3750,-2921648063749375980452588814336.0000,0.0600
auto_arima,Auto ARIMA,1.6584,1.0285,14.8909,16.4796,0.3847,0.3685,-142695164110158682766572519424.0000,2.2800
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.7145,1.0789,15.3167,17.2538,0.3749,0.3947,-29397802981633200238769471488.0000,0.2967
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.8143,1.1316,16.2359,18.1148,0.4044,0.4582,-60802106383591430129077714944.0000,0.4167
naive,Naive Forecaster,1.9847,1.2458,17.6806,19.9090,0.4190,0.5265,-2.2974,0.0400
croston,Croston,2.0043,1.1997,18.6039,19.5260,0.4732,0.4710,-3335998400044404642404055908352.0000,0.0300
snaive,Seasonal Naive Forecaster,2.0048,1.2541,17.8606,20.0421,0.4287,0.5415,-2.3280,0.0733
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,2.0306,1.2712,18.7229,20.6321,0.4125,0.4942,-2504847705733598406706716475392.0000,0.2233
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.0407,1.2754,18.3564,20.4681,0.4257,0.5058,-224211922454577839104573046784.0000,0.2433


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [2, 1]},
                                                   n_jobs=1)],
                    regressor=ExtraTreesRegressor(n_jobs=-1, random_state=123),
                    sp=2, window_length=2)


In [15]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
gasExp = TSForecastingExperiment()

gasExp.setup(data=gas_df, fh = 6, session_id = 123)

# Compare models
best_model = gasExp.compare_models()

print(best_model)
gasExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,altitude
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.5562,0.9775,13.9451,15.6457,0.3449,0.3379,-85003055041179295941413306368.0000,0.3433
grand_means,Grand Means Forecaster,1.6265,1.0137,15.1986,16.5317,0.4461,0.3750,-2921648063749375980452588814336.0000,0.0700
auto_arima,Auto ARIMA,1.6584,1.0285,14.8909,16.4796,0.3847,0.3685,-142695164110158682766572519424.0000,2.1733
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.7145,1.0789,15.3167,17.2538,0.3749,0.3947,-29397802981633200238769471488.0000,0.3167
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.8143,1.1316,16.2359,18.1148,0.4044,0.4582,-60802106383591430129077714944.0000,0.4900
naive,Naive Forecaster,1.9847,1.2458,17.6806,19.9090,0.4190,0.5265,-2.2974,0.0633
croston,Croston,2.0043,1.1997,18.6039,19.5260,0.4732,0.4710,-3335998400044404642404055908352.0000,0.0433
snaive,Seasonal Naive Forecaster,2.0048,1.2541,17.8606,20.0421,0.4287,0.5415,-2.3280,0.1000
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,2.0306,1.2712,18.7229,20.6321,0.4125,0.4942,-2504847705733598406706716475392.0000,0.1367
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.0407,1.2754,18.3564,20.4681,0.4257,0.5058,-224211922454577839104573046784.0000,0.1967


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [2, 1]},
                                                   n_jobs=1)],
                    regressor=ExtraTreesRegressor(n_jobs=-1, random_state=123),
                    sp=2, window_length=2)


In [16]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
pmExp = TSForecastingExperiment()

pmExp.setup(data=pm_df, fh = 6, session_id = 123)

# Compare models
best_model = pmExp.compare_models()

print(best_model)
pmExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,PM2_5
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.5424,0.2829,1.8920,2.0533,0.3018,0.2792,-0.7762,0.2700
croston,Croston,0.5614,0.2892,1.9787,2.1113,0.3226,0.2865,-0.2736,0.0467
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,0.5702,0.2952,2.0098,2.1472,0.3467,0.2890,-0.4802,0.2700
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.6016,0.3082,2.0281,2.1809,0.3202,0.3457,-4.4572,0.2400
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,0.6606,0.3491,2.2183,2.4743,0.3255,0.4148,-6.6946,0.2167
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.6768,0.3546,2.2769,2.5150,0.3349,0.4268,-6.7591,0.2033
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.6769,0.3546,2.2771,2.5153,0.3349,0.4269,-6.7594,0.1900
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.6783,0.3561,2.2816,2.5264,0.3351,0.4274,-6.8029,0.2567
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.6833,0.3582,2.2993,2.5414,0.3378,0.4310,-6.8328,0.2100
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.6833,0.3582,2.2993,2.5414,0.3378,0.4310,-6.8328,0.1733


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [2, 1]},
                                                   n_jobs=1)],
                    regressor=LGBMRegressor(n_jobs=-1, random_state=123), sp=2,
                    window_length=2)


In [17]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
ncExp = TSForecastingExperiment()

ncExp.setup(data=nc_df, fh = 6, session_id = 123)

# Compare models
best_model = ncExp.compare_models()

print(best_model)
ncExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,NC2_5
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.4570,0.2601,11.6847,14.0666,0.2321,0.2903,-3.2122,0.2067
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.4570,0.2601,11.6848,14.0667,0.2321,0.2903,-3.2122,0.1867
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.4571,0.2602,11.6866,14.0677,0.2321,0.2904,-3.2120,0.2667
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.4571,0.2602,11.6879,14.0675,0.2322,0.2904,-3.2118,0.1800
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.4571,0.2602,11.6879,14.0675,0.2322,0.2904,-3.2118,0.1867
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,0.4585,0.2597,11.7286,14.0449,0.2347,0.2920,-3.2001,0.1800
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.4610,0.2665,11.7960,14.4319,0.2304,0.2904,-3.2362,0.1933
polytrend,Polynomial Trend Forecaster,0.5726,0.3576,14.8740,19.6394,0.2705,0.3613,-3.8801,0.0233
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.6423,0.3427,17.5867,19.1272,0.4938,0.3800,-0.1490,0.1933
croston,Croston,0.6676,0.3437,18.1715,19.1811,0.4927,0.3954,-0.2670,0.0233


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [2, 1]},
                                                   n_jobs=1)],
                    regressor=LinearRegression(n_jobs=-1), sp=2,
                    window_length=2)


In [18]:
# import TSForecastingExperiment and init the class
from pycaret.time_series import TSForecastingExperiment
aqiExp = TSForecastingExperiment()

aqiExp.setup(data=aqi_df, fh = 6, session_id = 123)

# Compare models
best_model = aqiExp.compare_models()

print(best_model)
aqiExp.plot_model(best_model, plot = 'forecast', data_kwargs = {'fh' : 36})

,Description,Value
0,session_id,123
1,Target,AQI
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(63, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.3545,0.2349,4.5407,6.4517,0.1225,0.1355,-1.5939,0.2467
croston,Croston,0.3946,0.2255,5.1780,6.2377,0.1526,0.1537,-0.2925,0.0233
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.5387,0.3146,6.9224,8.5748,0.2054,0.2285,-3.8936,0.2633
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.6477,0.3449,8.5475,9.5565,0.2831,0.2433,-2.2191,0.2900
snaive,Seasonal Naive Forecaster,0.6733,0.3479,8.9444,9.7276,0.2972,0.2232,-2.3244,0.0900
naive,Naive Forecaster,0.6855,0.3607,9.1111,10.0916,0.3062,0.2256,-2.4815,0.0633
auto_arima,Auto ARIMA,0.6855,0.3607,9.1111,10.0916,0.3062,0.2256,-2.4815,0.8233
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.7145,0.3558,9.1762,9.6586,0.2974,0.3086,-5.7726,0.1667
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.7353,0.3886,9.6515,10.6943,0.3196,0.2878,-4.4521,0.4000
theta,Theta Forecaster,0.7396,0.3738,9.7024,10.3435,0.3208,0.2694,-3.8594,0.0467


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [2, 1]},
                                                   n_jobs=1)],
                    regressor=LGBMRegressor(n_jobs=-1, random_state=123), sp=2,
                    window_length=2)


In [19]:
# # Import necessary libraries
# from pycaret.time_series import TSForecastingExperiment
# import pandas as pd
# import requests
# import json

# # Define the URL of the endpoint
# url = "http://localhost/arduino/air-quality-asor/insert-aqi.php"

# # Initialize separate TSForecastingExperiment instances for each data type
# ncExp = TSForecastingExperiment()
# aqiExp = TSForecastingExperiment()
# pmExp = TSForecastingExperiment()
# tempExp = TSForecastingExperiment()
# humidExp = TSForecastingExperiment()
# pressureExp = TSForecastingExperiment()
# gasExp = TSForecastingExperiment()
# altitudeExp = TSForecastingExperiment()

# # Setup each experiment with respective data and forecast horizon
# ncExp.setup(data=nc_df, fh=12, session_id=123)
# aqiExp.setup(data=aqi_df, fh=12, session_id=123)
# pmExp.setup(data=pm_df, fh=12, session_id=123)
# tempExp.setup(data=temp_df, fh=12, session_id=123)
# humidExp.setup(data=humid_df, fh=12, session_id=123)
# pressureExp.setup(data=pressure_df, fh=12, session_id=123)
# gasExp.setup(data=gas_df, fh=12, session_id=123)
# altitudeExp.setup(data=altitude_df, fh=12, session_id=123)

# # Compare models for each data type
# best_nc_model = ncExp.compare_models()
# best_aqi_model = aqiExp.compare_models()
# best_pm_model = pmExp.compare_models()
# best_temp_model = tempExp.compare_models()
# best_humid_model = humidExp.compare_models()
# best_pressure_model = pressureExp.compare_models()
# best_gas_model = gasExp.compare_models()
# best_altitude_model = altitudeExp.compare_models()

# # Finalize the best model for each data type
# final_nc_model = ncExp.finalize_model(best_nc_model)
# final_aqi_model = aqiExp.finalize_model(best_aqi_model)
# final_pm_model = pmExp.finalize_model(best_pm_model)
# final_temp_model = tempExp.finalize_model(best_temp_model)
# final_humid_model = humidExp.finalize_model(best_humid_model)
# final_pressure_model = pressureExp.finalize_model(best_pressure_model)
# final_gas_model = gasExp.finalize_model(best_gas_model)
# final_altitude_model = altitudeExp.finalize_model(best_altitude_model)

# # Make predictions for each data type
# nc_predictions = ncExp.predict_model(final_nc_model)
# aqi_predictions = aqiExp.predict_model(final_aqi_model)
# pm_predictions = pmExp.predict_model(final_pm_model)
# temp_predictions = tempExp.predict_model(final_temp_model)
# humid_predictions = humidExp.predict_model(final_humid_model)
# pressure_predictions = pressureExp.predict_model(final_pressure_model)
# gas_predictions = gasExp.predict_model(final_gas_model)
# altitude_predictions = altitudeExp.predict_model(final_altitude_model)

# # Extract predicted values for each data type
# predicted_nc_values = nc_predictions['y_pred']
# predicted_aqi_values = aqi_predictions['y_pred']
# predicted_pm_values = pm_predictions['y_pred']
# predicted_temp_values = temp_predictions['y_pred']
# predicted_humid_values = humid_predictions['y_pred']
# predicted_pressure_values = pressure_predictions['y_pred']
# predicted_gas_values = gas_predictions['y_pred']
# predicted_altitude_values = altitude_predictions['y_pred']

# # Get the date index for the next 12 hours for each data type
# # Assuming the last timestamp in respective dataframes is the latest available timestamp
# last_nc_timestamp = nc_df.index[-1]
# last_aqi_timestamp = aqi_df.index[-1]
# last_pm_timestamp = pm_df.index[-1]
# last_temp_timestamp = temp_df.index[-1]
# last_humid_timestamp = humid_df.index[-1]
# last_pressure_timestamp = pressure_df.index[-1]
# last_gas_timestamp = gas_df.index[-1]
# last_altitude_timestamp = altitude_df.index[-1]

# next_nc_12_hours = pd.date_range(start=last_nc_timestamp, periods=12, freq='H').to_list()
# next_aqi_12_hours = pd.date_range(start=last_aqi_timestamp, periods=12, freq='H').to_list()
# next_pm_12_hours = pd.date_range(start=last_pm_timestamp, periods=12, freq='H').to_list()
# next_temp_12_hours = pd.date_range(start=last_temp_timestamp, periods=12, freq='H').to_list()
# next_humid_12_hours = pd.date_range(start=last_humid_timestamp, periods=12, freq='H').to_list()
# next_pressure_12_hours = pd.date_range(start=last_pressure_timestamp, periods=12, freq='H').to_list()
# next_gas_12_hours = pd.date_range(start=last_gas_timestamp, periods=12, freq='H').to_list()
# next_altitude_12_hours = pd.date_range(start=last_altitude_timestamp, periods=12, freq='H').to_list()

# # Combine date index with predicted values for each data type
# predicted_nc_data = pd.DataFrame({'Date': next_nc_12_hours, 'Predicted_Value': predicted_nc_values})
# predicted_aqi_data = pd.DataFrame({'Date': next_aqi_12_hours, 'Predicted_Value': predicted_aqi_values})
# predicted_pm_data = pd.DataFrame({'Date': next_pm_12_hours, 'Predicted_Value': predicted_pm_values})
# predicted_temp_data = pd.DataFrame({'Date': next_temp_12_hours, 'Predicted_Value': predicted_temp_values})
# predicted_humid_data = pd.DataFrame({'Date': next_humid_12_hours, 'Predicted_Value': predicted_humid_values})
# predicted_pressure_data = pd.DataFrame({'Date': next_pressure_12_hours, 'Predicted_Value': predicted_pressure_values})
# predicted_gas_data = pd.DataFrame({'Date': next_gas_12_hours, 'Predicted_Value': predicted_gas_values})
# predicted_altitude_data = pd.DataFrame({'Date': next_altitude_12_hours, 'Predicted_Value': predicted_altitude_values})

# # Convert predicted data to JSON format for each data type
# predicted_nc_data_json = predicted_nc_data.to_json(orient='records')
# predicted_aqi_data_json = predicted_aqi_data.to_json(orient='records')
# predicted_pm_data_json = predicted_pm_data.to_json(orient='records')
# predicted_temp_data_json = predicted_temp_data.to_json(orient='records')
# predicted_humid_data_json = predicted_humid_data.to_json(orient='records')
# predicted_pressure_data_json = predicted_pressure_data.to_json(orient='records')
# predicted_gas_data_json = predicted_gas_data.to_json(orient='records')
# predicted_altitude_data_json = predicted_altitude_data.to_json(orient='records')

# # Define the payload to be sent to the endpoint
# payload = {
#     'predicted_nc_data': predicted_nc_data_json,
#     'predicted_aqi_data': predicted_aqi_data_json,
#     'predicted_pm_data': predicted_pm_data_json,
#     'predicted_temp_data': predicted_temp_data_json,
#     'predicted_humid_data': predicted_humid_data_json,
#     'predicted_pressure_data': predicted_pressure_data_json,
#     'predicted_gas_data': predicted_gas_data_json,
#     'predicted_altitude_data': predicted_altitude_data_json
# }

# # Send POST request to the endpoint
# response = requests.post(url, data=json.dumps(payload))

# # Check if the request was successful
# if response.status_code == 200:
#     print("Predicted data inserted successfully.")
# else:
#     print("Error:", response.text)

# # Print predicted data for each data type
# print("Predicted NC Data:")
# print(predicted_nc_data)
# print("\nPredicted AQI Data:")
# print(predicted_aqi_data)
# print("\nPredicted PM Data:")
# print(predicted_pm_data)
# print("\nPredicted Temperature Data:")
# print(predicted_temp_data)
# print("\nPredicted Humidity Data:")
# print(predicted_humid_data)
# print("\nPredicted Pressure Data:")
# print(predicted_pressure_data)
# print("\nPredicted Gas Data:")
# print(predicted_gas_data)
# print("\nPredicted Altitude Data:")
# print(predicted_altitude_data)

# Import necessary libraries
from pycaret.time_series import TSForecastingExperiment
import pandas as pd
import requests
import json

# Define the URL of the endpoint
url = "https://mchaexpress.com/sensor_insert_predictions.php"

# Initialize separate TSForecastingExperiment instances for each data type
experiments = {
    "nc": TSForecastingExperiment(),
    "aqi": TSForecastingExperiment(),
    "pm": TSForecastingExperiment(),
    "temp": TSForecastingExperiment(),
    "humid": TSForecastingExperiment(),
    "pressure": TSForecastingExperiment(),
    "gas": TSForecastingExperiment(),
    "altitude": TSForecastingExperiment()
}

# Dataframes for each data type
dataframes = {
    "nc": nc_df,
    "aqi": aqi_df,
    "pm": pm_df,
    "temp": temperature_df,
    "humid": humidity_df,
    "pressure": pressure_df,
    "gas": gas_df,
    "altitude": altitude_df
}

# Setup each experiment with respective data and forecast horizon
for key, exp in experiments.items():
    exp.setup(data=dataframes[key], fh=12, session_id=123)

# Compare models for each data type
best_models = {key: exp.compare_models() for key, exp in experiments.items()}

# Finalize the best model for each data type
final_models = {key: exp.finalize_model(best_models[key]) for key, exp in experiments.items()}

# Make predictions for each data type
predictions = {key: exp.predict_model(final_models[key]) for key, exp in experiments.items()}

# Extract predicted values for each data type
predicted_values = {key: pred['y_pred'] for key, pred in predictions.items()}

# Get the date index for the next 12 hours for each data type
last_timestamps = {key: df.index[-1] for key, df in dataframes.items()}
next_12_hours = {key: pd.date_range(start=last_timestamps[key], periods=12, freq='H').to_list() for key in dataframes}

# Combine date index with predicted values for each data type
predicted_data = {key: pd.DataFrame({'Date': next_12_hours[key], 'Predicted_Value': predicted_values[key]}) for key in dataframes}

# Convert predicted data to JSON format for each data type
predicted_data_json = {key: data.to_json(orient='records') for key, data in predicted_data.items()}

# Define the payload to be sent to the endpoint
payload = {f'predicted_{key}_data': value for key, value in predicted_data_json.items()}

# Send POST request to the endpoint
response = requests.post(url, data=json.dumps(payload))

# Check if the request was successful
if response.status_code == 200:
    print("Predicted data inserted successfully.")
else:
    print("Error:", response.text)

# Print predicted data for each data type
for key, data in predicted_data.items():
    print(f"Predicted {key.capitalize()} Data:")
    print(data)

,Description,Value
0,session_id,123
1,Target,NC2_5
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(57, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,123
1,Target,AQI
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(57, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,123
1,Target,PM2_5
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(57, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,123
1,Target,temperature
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(57, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,123
1,Target,humidity
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(69, 1)"
5,Transformed data shape,"(69, 1)"
6,Transformed train set shape,"(57, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


...

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,1.1860,0.9455,18.7417,26.2861,0.2690,0.1975,-4681.4143,1.3700
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.5513,1.1375,23.9750,31.3762,0.3245,0.2493,-6396.8319,0.2700
naive,Naive Forecaster,1.6622,1.0671,26.7614,29.5187,0.4223,0.2807,-8837.6203,0.1033
stlf,STLF,1.7153,1.0988,26.9422,31.1252,0.3535,0.3574,-3353.8543,0.0533
arima,ARIMA,1.7591,1.2209,27.5892,34.4211,0.3619,0.3436,-4620.4502,0.0833
snaive,Seasonal Naive Forecaster,1.7702,1.2689,26.9925,35.1474,0.3416,0.2994,-6478.9135,0.0733
ets,ETS,1.8384,1.1291,29.4778,32.0167,0.4200,0.3884,-4172.6596,0.0867
theta,Theta Forecaster,1.9485,1.1812,31.1145,33.3166,0.4445,0.3947,-5090.8934,0.0567
grand_means,Grand Means Forecaster,2.2271,1.2695,35.5704,35.8269,0.5195,0.4047,-5661.0088,0.0367
croston,Croston,2.2542,1.2808,34.8719,35.4977,0.4706,0.4019,-6654.8983,0.0267


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,1.7336,0.9524,368.0228,437.4517,0.3497,0.2425,-823942.1959,0.0667
ets,ETS,1.8272,0.9872,381.8752,450.1515,0.3544,0.2553,-759483.6727,0.0633
exp_smooth,Exponential Smoothing,1.9103,0.9999,395.5477,455.0942,0.3617,0.2676,-727634.1665,0.1000
theta,Theta Forecaster,2.1172,1.0703,429.4955,482.8406,0.3814,0.3138,-591013.7100,0.0433
arima,ARIMA,2.6854,1.3253,531.3086,591.4574,0.4486,0.3758,-713112.6598,0.0933
snaive,Seasonal Naive Forecaster,2.7451,1.5304,535.3667,673.4512,0.4325,0.3530,-876808.1880,0.0567
stlf,STLF,2.9058,1.4550,578.0013,649.9870,0.4935,0.3939,-966050.2760,0.0533
auto_arima,Auto ARIMA,3.0082,1.4307,579.9253,631.0998,0.4674,0.4485,-523042.8200,1.8700
croston,Croston,3.4319,1.5089,643.7387,657.1727,0.4873,0.4428,-602348.8672,0.0300
grand_means,Grand Means Forecaster,3.4488,1.5074,664.9120,665.0897,0.5557,0.4511,-637937.8135,0.0367


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,1.3737,0.9628,14.3420,17.0742,0.5960,0.4629,-1.9297,1.6133
grand_means,Grand Means Forecaster,1.3752,0.9427,14.6445,16.9380,0.7399,0.4324,-2.1200,0.0367
croston,Croston,1.4482,1.0115,14.6702,17.7145,0.6464,0.4499,-1.8155,0.0433
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.6120,1.0300,15.5828,17.5710,0.5151,0.5068,-1.6316,0.2233
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.6130,1.0335,15.6229,17.6604,0.5161,0.5071,-1.6782,0.2367
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.6130,1.0335,15.6229,17.6606,0.5161,0.5071,-1.6783,0.2400
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.6137,1.0336,15.6268,17.6585,0.5161,0.5073,-1.6762,0.2700
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.6140,1.0335,15.6283,17.6567,0.5161,0.5074,-1.6748,0.2300
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.6140,1.0335,15.6283,17.6567,0.5161,0.5074,-1.6748,0.2333
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.6185,1.0169,15.4667,17.1572,0.5008,0.5073,-1.4458,0.2300


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,1.3737,0.9628,14.3420,17.0742,0.5960,0.4629,-1.9297,1.5767
grand_means,Grand Means Forecaster,1.3752,0.9427,14.6445,16.9380,0.7399,0.4324,-2.1200,0.0333
croston,Croston,1.4482,1.0115,14.6702,17.7145,0.6464,0.4499,-1.8155,0.0267
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.6120,1.0300,15.5828,17.5710,0.5151,0.5068,-1.6316,0.2233
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.6130,1.0335,15.6229,17.6604,0.5161,0.5071,-1.6782,0.2867
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.6130,1.0335,15.6229,17.6606,0.5161,0.5071,-1.6783,0.2167
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.6137,1.0336,15.6268,17.6585,0.5161,0.5073,-1.6762,0.2733
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.6140,1.0335,15.6283,17.6567,0.5161,0.5074,-1.6748,0.2167
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.6140,1.0335,15.6283,17.6567,0.5161,0.5074,-1.6748,0.2400
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.6185,1.0169,15.4667,17.1572,0.5008,0.5073,-1.4458,0.2400


Predicted data inserted successfully.
Predicted Nc Data:
                                Date  Predicted_Value
2024-04-09 03:00 2024-04-09 02:00:00          75.1174
2024-04-09 04:00 2024-04-09 03:00:00          75.1174
2024-04-09 05:00 2024-04-09 04:00:00          75.1174
2024-04-09 06:00 2024-04-09 05:00:00          75.1174
2024-04-09 07:00 2024-04-09 06:00:00          75.1174
2024-04-09 08:00 2024-04-09 07:00:00          75.1174
2024-04-09 09:00 2024-04-09 08:00:00          75.1174
2024-04-09 10:00 2024-04-09 09:00:00          75.1174
2024-04-09 11:00 2024-04-09 10:00:00          75.1174
2024-04-09 12:00 2024-04-09 11:00:00          75.1174
2024-04-09 13:00 2024-04-09 12:00:00          75.1174
2024-04-09 14:00 2024-04-09 13:00:00          75.1174
Predicted Aqi Data:
                                Date  Predicted_Value
2024-04-09 03:00 2024-04-09 02:00:00           46.338
2024-04-09 04:00 2024-04-09 03:00:00           46.338
2024-04-09 05:00 2024-04-09 04:00:00           46.338
2024-